<a href="https://colab.research.google.com/github/MrZombie69232/jovian.ml/blob/master/rebase_pvlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# load pvlib required packages
import os

import itertools

import matplotlib.pyplot as plt

import pandas as pd


In [2]:
pip install pvlib

     |████████████████████████████████| 27.5 MB 111 kB/s 


In [3]:
import pvlib

from pvlib import clearsky, atmosphere, solarposition

from pvlib.location import Location

from pvlib.iotools import read_tmy3

In [4]:
# load geolocation packages
from geopy.geocoders import Nominatim
import time
from pprint import pprint
import requests
import pytz


In [5]:
pip install Nominatim

  Created wheel for Nominatim: filename=nominatim-0.1-py3-none-any.whl size=2363 sha256=65631a821b1ccc7e051baeace92d13368acdbcc8161d5b78a999902662dd6c1b
  Stored in directory: /root/.cache/pip/wheels/37/00/9e/d904c390bfb174830ad3dcfd62af5544cee7d88bed4f8acedd
Successfully built Nominatim


In [6]:
import pandas as pd

In [7]:
def get_elevation(lat, long):
    query = ('https://api.open-elevation.com/api/v1/lookup'
             f'?locations={lat},{long}')
    r = requests.get(query).json()  # json object, various ways you can extract value
    # one approach is to use pandas json functionality:
    elevation = pd.io.json.json_normalize(r, 'results')['elevation'].values[0]
    return elevation

In [8]:
def local_hour_angle(apparent_sidereal_time, observer_longitude,
                     sun_right_ascension):
    """Measured westward from south"""
    print(f'{apparent_sidereal_time!r}')
    print(f'{observer_longitude!r}')
    print(f'{sun_right_ascension!r}')
    H = apparent_sidereal_time + observer_longitude - sun_right_ascension
    return H % 360

In [9]:
pip install pvlib

In [10]:
from pvlib.location import Location

In [11]:
import pvlib
if __name__ == '__main__':
    app = Nominatim(user_agent="test_solar")

    loc_raw = app.geocode('Postintaival 7, 00230 Helsinki, Finland').raw

    latitude = loc_raw['lat']
    longitude = loc_raw['lon']
    latitude = float(loc_raw['lat'])
    longitude = float(loc_raw['lon'])
    altitude = get_elevation(latitude, longitude)

    location_object = Location(latitude, longitude, 'Europe/Helsinki',
                               altitude, 'relex_solutions')
    weather = pvlib.iotools.get_pvgis_tmy(latitude, longitude,
                                          map_variables=True)[0]

    times = weather.index

    solpos = location_object.get_solarposition(times)
    clearsky_values = location_object.get_clearsky(times, model='ineichen',
                                                   solar_position=solpos, dni_extra=None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [17]:
df = pd.DataFrame(clearsky_values)
df.to_csv('clearsky_values.csv')

In [13]:
df.columns

Index(['ghi', 'dni', 'dhi'], dtype='object')

In [14]:
df.describe

<bound method NDFrame.describe of                            ghi  dni  dhi
time(UTC)                               
2012-01-01 00:00:00+00:00  0.0  0.0  0.0
2012-01-01 01:00:00+00:00  0.0  0.0  0.0
2012-01-01 02:00:00+00:00  0.0  0.0  0.0
2012-01-01 03:00:00+00:00  0.0  0.0  0.0
2012-01-01 04:00:00+00:00  0.0  0.0  0.0
...                        ...  ...  ...
2014-12-31 19:00:00+00:00  0.0  0.0  0.0
2014-12-31 20:00:00+00:00  0.0  0.0  0.0
2014-12-31 21:00:00+00:00  0.0  0.0  0.0
2014-12-31 22:00:00+00:00  0.0  0.0  0.0
2014-12-31 23:00:00+00:00  0.0  0.0  0.0

[8760 rows x 3 columns]>

In [16]:
df2 = pd.DataFrame(solpos)
df2.to_csv('solpos.csv')